### Libraries Used

In [3]:
import ollama

### Understanding how to generate reponses using Ollama LLMs

In [4]:
# 1. Define the question
user_question = "What is the capital of India?"

# 2. Send it to the model (Generation only)
# We are NOT providing external context yet.
response = ollama.chat(
    model='gemma3:4b',  # Enter the model that you pull on ollama
    messages=[{'role': 'user', 'content': user_question}]
)

# 3. Print the answer
print("AI Answer:", response['message']['content'])

AI Answer: The capital of India is **New Delhi**. 

It's a planned city, officially named Delhi when it was founded in 1911 as the new capital, replacing Calcutta.


We write,<br>
`response['message']['content']`<br>
because it is in JSON format:<br>

```json
{
  "model": "phi3:3.8b",
  "created_at": "2025-12-15T09:45:00.5105742Z",
  "done": true,
  "done_reason": "stop",
  "total_duration": 11129766800,
  "load_duration": 3728855400,
  "prompt_eval_count": 16,
  "prompt_eval_duration": 765201100,
  "eval_count": 94,
  "eval_duration": 6592721000,
  "message": {
    "role": "assistant",
    "content": "The capital of India is New Delhi. It became the political center in 1956 when the country's government moved from Kolkata (then called Calcutta) and now serves as a major hub for culture, education, and politics, housing Parliament House, which holds the Indian legislative bodies - Lok Sabha (House of the People), Rajya Sabha (Council of States), along with numerous other government offices.",
    "thinking": null,
    "images": null,
    "tool_name": null,
    "tool_calls": null
  },
  "logprobs": null
}



### Different Roles

| Role      | Analogy               | Purpose                                  |
|-----------|-----------------------|------------------------------------------|
| System    | The Boss / Director   | Defines behavior, tone, and rules.        |
| User      | The Customer          | Asks the question.                       |
| Assistant | The Transcript        | Provides conversation history/context.   |


```py
messages=[
    # 1. We set the context (optional but good)
    {'role': 'system', 'content': 'You are a helpful assistant.'},

    # 2. First turn
    {'role': 'user', 'content': 'Who wrote Harry Potter?'},
    {'role': 'assistant', 'content': 'J.K. Rowling wrote it.'}, # <--- WE INSERT THIS MANUALLY OR FROM PREVIOUS RESPONSE

    # 3. Current question (The AI now reads the line above and knows 'she' = Rowling)
    {'role': 'user', 'content': 'What year was she born?'}
]

### Chunking

In [5]:
# 1. The text we want to translate
text = "The cat watched carefully as the playful kitten chased a leaf across the driveway, only stopping when a car rolled slowly past the house."


# 2. Ask Ollama to create the embedding
# We use 'nomic-embed-text' because it's built for this.
response = ollama.embeddings(
    model='nomic-embed-text',
    prompt=text
)

# 3. Get the vector (The list of numbers)
vector = response['embedding']

# 4. Inspect the result
print(f"Text: {text}")
print(f"Vector Length: {len(vector)}") # How many dimensions?
print(f"First 5 numbers: {vector[:5]}") # Just a peek

Text: The cat watched carefully as the playful kitten chased a leaf across the driveway, only stopping when a car rolled slowly past the house.
Vector Length: 768
First 5 numbers: [1.0189101696014404, 0.19626553356647491, -2.8098843097686768, 0.09355314821004868, 1.6594434976577759]


### Vector Stores

In [6]:
import chromadb

# 1. Initialize the Client (The Librarian)
# This creates a temporary in-memory database. 
# (Use persistent_client to save to disk, but for now, memory is fine)
client = chromadb.Client()

# 2. Create a Collection (The Bookshelf)
# A 'collection' is where we store related documents.
collection = client.create_collection(name="dragon_facts") #here since we didn't add any model, so it will download a default embedding model

# 3. Add Documents (The Knowledge)
# We give each chunk of text an ID so we can find it later.
documents = [
    "My dragon's name is Fluffy.",
    "Fluffy eats 500kg of spicy tacos every Tuesday.",
    "Fluffy lives in a volcano behind the Walmart."
]
ids = ["id1", "id2", "id3"]

# 4. Store them
# NOTICE: Chroma handles the embedding part automatically in the background!
collection.add(
    documents=documents,
    ids=ids
)

print("Library built! I currently know", collection.count(), "facts about the dragon.")

Library built! I currently know 3 facts about the dragon.


### Retrival

In [7]:
# 1. Setup (Same as before)
client = chromadb.Client()
collection = client.create_collection(name="dragon_facts_test")

documents = [
    "My dragon's name is Fluffy.",
    "Fluffy eats 500kg of spicy tacos every Tuesday.",
    "Fluffy lives in a volcano behind the Walmart."
]
collection.add(documents=documents, ids=["id1", "id2", "id3"])

# ---------------------------------------------------------
# 2. The Retrieval Step
# ---------------------------------------------------------

query_text = "Tell me about the creature's diet"

results = collection.query(
    query_texts=[query_text],
    n_results=1  # How many top matches do you want? Let's ask for the best ONE.
)

# 3. Inspect the results
print("Question:", query_text)
print("\n--- Retrieved Document ---")
print(results['documents'][0][0]) # It returns a list of lists, so we grab the first item.

Question: Tell me about the creature's diet

--- Retrieved Document ---
Fluffy eats 500kg of spicy tacos every Tuesday.
